In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from laplace_equation.utils.result_generation import build_input_matrix

In [2]:
data = pd.read_csv("laplace_equation/data/result.csv")
data

,nominal_temperature,boundary_conditions,result
0,92,"[2, 43, 24, 80, 63, 43, 23, 78, 73, 72, 20, 85...",[[ 2. 43. 24. 80. ...
1,91,"[60, 52, 19, 0, 28, 6, 24, 67, 24, 66, 58, 30,...",[[60. 52. 19. 0. ...
2,27,"[57, 24, 31, 77, 8, 96, 5, 86, 95, 43, 59, 58,...",[[57. 24. 31. 77. ...
3,36,"[65, 77, 61, 71, 48, 79, 32, 91, 37, 84, 69, 7...",[[65. 77. 61. 71. ...
4,25,"[30, 45, 40, 56, 42, 0, 13, 28, 83, 2, 53, 5, ...",[[30. 45. 40. 56. ...
...,...,...,...
8995,30,"[98, 98, 98, 98, 98, 98, 98, 98, 98, 27, 27, 2...",[[98. 98. 98. 98. ...
8996,100,"[41, 41, 41, 41, 41, 41, 41, 41, 41, 30, 30, 3...",[[41. 41. 41. 41. ...
8997,20,"[39, 39, 39, 39, 39, 39, 39, 39, 39, 83, 83, 8...",[[39. 39. 39. 39. ...
8998,55,"[43, 43, 43, 43, 43, 43, 43, 43, 43, 66, 66, 6...",[[43. 43. 43. 43. ...


In [3]:
input_data = []
for i, record in data.iterrows():
    input_temperature = build_input_matrix(record["nominal_temperature"], record["boundary_conditions"])
    input_data.append(input_temperature)    
    
input_data = np.asarray(input_data)
output_data = np.load("laplace_equation/data/output_data.npy")

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(input_data.reshape(-1, 100), output_data.reshape(-1, 100), test_size=0.2, random_state=42)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [6]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.fc1 = Dense(32, activation='relu')
    self.fc2 = Dense(128, activation='relu')
    self.fc3 = Dense(256, activation='relu')
    self.fc4 = Dense(100)

  def call(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    return x

# Create an instance of the model
model = MyModel()
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Accuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.Accuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(input_data, output_data):
  with tf.GradientTape() as tape:
    predictions = model(input_data, training=True)
    loss = loss_fn(output_data, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(tf.round(output_data), tf.round(predictions))

In [8]:
@tf.function
def test_step(input_data, output_data):
  predictions = model(input_data, training=False)
  t_loss = loss_fn(output_data, predictions)

  test_loss(t_loss)
  test_accuracy(tf.round(output_data), tf.round(predictions))

In [9]:
nb_epoch = 80

for epoch in range(nb_epoch):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for input_temperature, output_temperature in train_dataset:
    train_step(input_temperature, output_temperature)

  for input_temperature, output_temperature in test_dataset:
    test_step(input_temperature, output_temperature)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 254.18267822265625, Accuracy: 7.7384724617004395, Test Loss: 42.54594802856445, Test Accuracy: 15.328332901000977
Epoch 2, Loss: 30.422178268432617, Accuracy: 18.850555419921875, Test Loss: 19.511070251464844, Test Accuracy: 20.91777801513672
Epoch 3, Loss: 16.354156494140625, Accuracy: 27.651803970336914, Test Loss: 12.162492752075195, Test Accuracy: 33.56111145019531
Epoch 4, Loss: 12.230454444885254, Accuracy: 32.4627799987793, Test Loss: 9.976943016052246, Test Accuracy: 32.051109313964844
Epoch 5, Loss: 10.00681209564209, Accuracy: 35.84291458129883, Test Loss: 7.989902973175049, Test Accuracy: 36.42277526855469
Epoch 6, Loss: 8.499279975891113, Accuracy: 38.45527648925781, Test Loss: 6.772518157958984, Test Accuracy: 44.15055465698242
Epoch 7, Loss: 7.565160751342773, Accuracy: 40.01583480834961, Test Loss: 6.476013660430908, Test Accuracy: 35.078887939453125
Epoch 8, Loss: 6.9610595703125, Accuracy: 39.649166107177734, Test Loss: 5.687795162200928, Test Accuracy: 